# Set Membership Coursework

In [ ]:
%pip install bitarray

In [ ]:
import random
import string
import timeit
from bitarray import bitarray

## 1. Implementeation

### Sequential Search 

In [ ]:
class SequentialSearchSet():
    
    def __init__(self):
        self.items = []           
        
    def insertElement(self, element):
        inserted = False
        if type(element) == str:
            self.items.append(element)
            inserted = True
        return inserted   

    def searchElement(self, element):
        found = False     
        for item1 in self.items:
            if item1 == element:
                found = True
                return found
        return found   

### Binary Search Tree

In [ ]:
class RecursiveNode:
    
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

    def get(self, element):
        if self.value == element:
            return True
        elif element < self.value and self.left:
            return self.left.get(element)
        elif element > self.value and self.right:
            return self.right.get(element)
        return False

    def put(self, element):
        if self.value == None:
            self.value = element
            return True
        elif element == self.value:
            self.value = element
            return True
        elif element < self.value:
            if self.left is None:
                self.left = RecursiveNode(element)
                return True
            else: 
                return self.left.put(element)
        elif element > self.value:
            if self.right is None:
                self.right = RecursiveNode(element)
                return True
            else: 
                return self.right.put(element)
        return False

class IterativeNode:
    
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

    def get(self, element):
        current = self
        while current is not None:
            if current.value == element:
                return True
            elif element < current.value:
                current = current.left
            elif element > current.value:
                current = current.right
            else:
                print("Invalid Case")
        return False


    def put(self, element):
        if self.value == None:
            self.value = element
            return True
        
        current = self
        while current is not None:
            if element < current.value:
                if current.left is None:
                    current.left = IterativeNode(element)
                    return True
                else:
                    current = current.left

            elif element > current.value:
                if current.right is None:
                    current.right = IterativeNode(element)
                    return True
                else:
                    current = current.right

            elif element == current.value:
                current.value = element
                return True
            
            else:
                print("Invalid Case")


class BinarySearchTreeSet:
    
    def __init__(self):
        self.bst = IterativeNode(None)

    def insertElement(self, element):
        inserted = False
        inserted = self.bst.put(element) 
        return inserted

    def searchElement(self, element):     
        found = False 
        found = self.bst.get(element)
        return found

### Balanced Search Tree

In [ ]:
class BalancedSearchTreeSet:
    R = True
    B = False
    # R refers to Red; B refers to Black
    
    class Nodes:
        def __init__(self, val, colour):
            self.val = val
            self.colour = colour
            self.left = None
            self.right = None
            
    def __init__(self):
        self.root = None
        
    def searchElement(self, element):
        found = False
        found = self.search(self.root, element)
        return found
    
    def search(self, node, val):
        if node is None:
            return False
        else:
            if node.val == val:
                return True
            elif node.val > val:
                return self.search(node.left, val)
            else:
                return self.search(node.right, val)
    
    def insertElement(self, element):
        inserted = False
        if self.root is None:
            self.root = self.Nodes(element, self.B)
            inserted = True
        else:
            self.root, inserted = self.insert(self.root, element)
            self.root.colour = self.B
        return inserted

    def insert(self, node, val):
        if node is None:
            return self.Nodes(val, self.R), True
        if val < node.val:
            node.left, inserted = self.insert(node.left, val)
        elif val > node.val:
            node.right, inserted = self.insert(node.right, val)
        else:
            # value already exists, do nothing
            return node, False

        # perform rotations and colour flips
        if self.isR(node.right) and not self.isR(node.left):
            node = self.leftRotate(node)
        if self.isR(node.left) and self.isR(node.left.left):
            node = self.rightRotate(node)
        if self.isR(node.left) and self.isR(node.right):
            self.flipColour(node)

        return node, inserted

    def isR(self, node):
        if node is None:
            return False
        return node.colour == self.R
    
    def leftRotate(self, node):
        x = node.right
        node.right = x.left
        x.left = node
        x.colour = node.colour
        node.colour = self.R
        return x
    
    def rightRotate(self, node):
        x = node.left
        node.left = x.right
        x.right = node
        x.colour = node.colour
        node.colour = self.R
        return x
    
    def flipColour(self, node):
        node.colour = not node.colour
        node.left.colour = self.B
        node.right.colour = self.B

### 2-3 Search Tree

In [ ]:
class Node2:
    def __init__(self, values, parent = None):
        self.values = [values]
        self.parent = parent
        self.child = []
    
    def is_leaf(self):
        return len(self.child) == 0
    
    def __lt__(self, other):
        return self.values[0] < other.values[0]
    
    def add(self, new_node):
        for child in new_node.child:
            child.parent = self
        self.values.extend(new_node.values)
        self.values.sort()
        self.child.extend(new_node.child)
        if len(self.child)>1:
            self.child.sort()
        if len(self.values)>2:
            self.split()
    
    def _insert(self, new_node):
        if self.is_leaf() == True:
            if new_node.values[0] not in self.values:
                self.add(new_node)
                return True
            return False
        elif new_node.values[0] > self.values[-1]:
            return self.child[-1]._insert(new_node)
        else:
            for i in range(0, len(self.values)):
                if new_node.values[0] < self.values[i]:
                    return self.child[i]._insert(new_node)
                    
    
    def split(self):
        left_child = Node2(self.values[0], self)
        right_child = Node2(self.values[2], self)
        if len(self.child) != 0:
            self.child[0].parent = left_child
            self.child[1].parent = left_child
            self.child[2].parent = right_child
            self.child[3].parent = right_child
            left_child.child = [self.child[0], self.child[1]]
            right_child.child = [self.child[2], self.child[3]]
        
        #promote middle node
        self.child = [left_child]
        self.child.append(right_child)
        self.values = [self.values[1]]

        if self.parent is not None:
            if self in self.parent.child:
                self.parent.child.remove(self)
            self.parent.add(self)
        else:
            left_child.parent = self
            right_child.parent = self

    def _find(self, element):
        if element in self.values:
            return True
        elif self.is_leaf() == True:
            return False
        elif element > self.values[-1]:
            return self.child[-1]._find(element)
        else:
            for i in range (0, len(self.values)):
                if element < self.values[i]:
                    return self.child[i]._find(element)
        
class TwoThreeBSTSet:
    def __init__(self):
        self.root = None
    
    def insertElement(self, element):
        inserted = False
        new_node = Node2(element)
        if self.root is None:
            self.root = new_node
        else:
            inserted = self.root._insert(new_node)
            if not inserted:
                return False
            #re-establishes the root to the top of the tree
            while self.root.parent is not None:
                self.root = self.root.parent
        return True 
    
    def searchElement(self, element):
        found = False
        found =self.root._find(element)
        return found

### Bloom Filter

In [ ]:
class BloomFilterSet:
    
    def __init__(self):
        self.HashFunctions = None
        self.hashNum = 0
        self.size = 0
        self.bitArray = None
    
    def initVariables(self, ItemNum, FPProb, HashFunctions):
        # ItemNum : int, Number of items expected to be stored in bloom filter
        # FPProb : float, False Positive probability in decimal
        # HashFunctions: list, number of hash functions to use
        self.HashFunctions = HashFunctions
        self.hashNum = len(HashFunctions)
        # Total size of bit array
        self.size = self.getSize(ItemNum, FPProb)
        # Bit array of given size, and initialize all bits as 0 (False)
        self.bitArray = bitarray(self.size)
        self.bitArray.setall(0)

    def insertElement(self, element):
        inserted = False
        tempIndex = []
        for function in self.HashFunctions:
            index = function(element) % self.size
            self.bitArray[index] = True
            tempIndex.append(self.bitArray[index])
        if False not in tempIndex:
            inserted = True
        return inserted

    def searchElement(self, element):
        found = False
        count = 0
        for function in self.HashFunctions:
            index = function(element) % self.size
            if not self.bitArray[index] == False:
                count += 1
            else:
                return found
        if count == len(self.HashFunctions):
            found = True
        return found

    def getSize(self, n, p):
        # m = -(n * lg(p)) / (lg(2)^2)
        # n : number of items expected to be stored in filter
        # p : False Positive probability in decimal
        m = -(n * self.myLog(p)) / (self.myLog(2) ** 2)
        return int(m)
    
    def myLog(self, n):
        if n <= 0:
            raise ValueError("logarithm is undefined for non-positive values")
        if n == 1:
            return 0
        if n < 1:
            # log(xn) = -log(1/n) for n < 1
            return -self.myLog(1 / n)
        # Use the taylor series approximation for log(n) where |n-1| < 1
        taylor = (n - 1) / float(n + 1)
        term = taylor
        total = taylor
        num = 1
        while abs(term) > 1e-15:
            num += 2
            term *= taylor * taylor / float(num * (num - 1))
            total += term
        return 2 * total

In [ ]:
def builtinHash(string):
    return hash(string)

def fnvHash(string):
    hashValue = 2166136261
    for char in string:
        hashValue = (hashValue * 16777619) ^ ord(char)
    return hashValue

def jenkinsHash(string):
    hashValue = 0
    for char in string:
        hashValue += ord(char)
        hashValue += (hashValue << 10)
        hashValue ^= (hashValue >> 6)
    hashValue += (hashValue << 3)
    hashValue ^= (hashValue >> 11)
    hashValue += (hashValue << 15)
    return hashValue

def sdbmHash(string):
    hashValue = 0
    for char in string:
        hashValue = ord(char) + (hashValue << 6) + (hashValue << 16) - hashValue
    return hashValue

def fnv1aHash(string):
    hashValue = 2166136261
    for char in string:
        hashValue ^= ord(char)
        hashValue *= 16777619
    return hashValue

## 2. Experimentally Evaluation

### Using Real Data

In [ ]:
def ReadFile(filepath):
    with open(filepath) as f:
        for line in f:
            words = line.strip().split()
    return words

In [ ]:
words1 = ReadFile("./data/testfiles/test1-mobydick.txt")
words2 = ReadFile("./data/testfiles/test2-warpeace.txt")
words3 = ReadFile("./data/testfiles/test3-dickens.txt")

files = {"'test1-mobydick.txt'": words1, "'test2-warpeace.txt'": words2,
         "'test3-dickens.txt'": words3}
print(len(words1), len(words2), len(words3))

In [ ]:
wordsforsearch = []
with open("./data/testfiles/test-search.txt") as fsearch:
    for line in fsearch:
        wordsforsearch.append(line.strip())
        
len(wordsforsearch)

In [ ]:
hashFunctions = [builtinHash, jenkinsHash, fnv1aHash, fnvHash, sdbmHash]

In [ ]:
def CheckRes(reslist):
    return all(all(x == reslist[0][i] for x in lst) for i, lst in enumerate(zip(*reslist)))

#### The total amount of time of all insert operations

In [ ]:
def InsertTime(classname, datadict):
    res = []
    classlist = []
    
    for key in datadict.keys():
        tempdata = datadict[key]
        algorithm = classname()
        if str(classname) == str(BloomFilterSet):
            algorithm.initVariables(len(tempdata), 0.01, hashFunctions)

        seen = {}
        InsertStart = timeit.default_timer()
        for word in tempdata:
            if word is not None and word not in seen:
                seen[word] = True
                inserted = algorithm.insertElement(word)
                res.append(inserted)
        InsertEnd = timeit.default_timer()
        InsertTime = InsertEnd - InsertStart
        print("Insert time taken for", key, "is :", InsertTime)
        classlist.append(algorithm)
    return classlist, res

In [ ]:
# Sequential Search
SEQs, SeqInsertRes = InsertTime(SequentialSearchSet, files)

In [ ]:
# Binary Search Tree
BSTs, BstInsertRes = InsertTime(BinarySearchTreeSet, files)

In [ ]:
# Red Black Balanced Search Tree
RBTs, BalstInsertRes = InsertTime(BalancedSearchTreeSet, files)

In [ ]:
# 2-3 Balanced Search Tree
TTSTs, TtstInsertRes = InsertTime(TwoThreeBSTSet, files)

In [ ]:
# Bloom Filter
BFs, BfInsertRes = InsertTime(BloomFilterSet, files)

In [ ]:
RealInsertList = [SeqInsertRes, BstInsertRes, BalstInsertRes, TtstInsertRes, BfInsertRes]
RealInsertRes = CheckRes(RealInsertList)
RealInsertRes

#### The total amount of time of search operations

In [ ]:
def SearchTime(objects, searchdata):
    res = []
    for obj in objects:
        tempres = []
        SearchStart = timeit.default_timer()
        for ws in searchdata:
            if ws is not None:
                searched = obj.searchElement(ws)
            else:
                searched = None
            tempres.append(searched)
        SearchEnd = timeit.default_timer()
        SearchTime = SearchEnd - SearchStart
        print("Search time taken for 'test-search.txt' in file", 
              objects.index(obj)+1, "is :", SearchTime)
        res.append(tempres)
    return res

In [ ]:
# Sequential Search Tree
SeqSearchRes = SearchTime(SEQs, wordsforsearch)

In [ ]:
# Binary Search Tree
BstSearchRes = SearchTime(BSTs, wordsforsearch)

In [ ]:
# Red Black Balanced Search Tree
BalstSearchRes = SearchTime(RBTs, wordsforsearch)

In [ ]:
# 2-3 Balanced Search Tree
TtstSearchRes = SearchTime(TTSTs, wordsforsearch)

In [ ]:
# Bloom Filter
BfSearchRes = SearchTime(BFs, wordsforsearch)

In [ ]:
RealSearchList = [SeqSearchRes, BstSearchRes, BalstSearchRes, TtstSearchRes, BfSearchRes]
RealSearchRes = CheckRes(RealSearchList)
RealSearchRes

In [ ]:
# The RealSearchRes is True, so the False-Positive Rate of Bloom filter is 0.0

### Using Synthetic Data

### different conditions:
- Duplicate cases
- Longer String length
- Adding numbers and symbols in each string
- String list in ascending/descending order
- Missing values 
- Strings of distinct integer values 
- Strings of distinct floats between 0.0 and 1.0
- Strings that contain whitespace
- Palindrome Strings
- Strings with highly variable lengths
- Strings with repeating patterns 
- Strings with alternating upper-case and lower-case letters

In [ ]:
class TestDataGenerator():

    def __init__(self):
        self.syntheticData = []
        self.strLen = 0
        self.percentage = 0.0

    def setParams(self, strLen, percentage):
        self.strLen = strLen
        self.percentage = percentage
    
    def generateData(self, size):
        data, data1, data2 = [], [], []
        seen1, seen2, seen3 = {}, {}, {}
        alphabets = list(string.ascii_letters)
        charlist = string.printable

        # Normal Case
        while len(data) < size:
            temp = ''.join(random.choice(alphabets) for _ in range(random.randint(4, 10)))
            if temp not in seen1:
                seen1[temp] = True
                data.append(temp)

        # Long String
        while len(data1) < size:
            temp1 = ''.join(random.choice(alphabets) for _ in range(self.strLen))
            if temp1 not in seen2:
                seen2[temp1] = True
                data1.append(temp1)

        # Special String
        while len(data2) < size:
            temp2 = ''.join(random.sample(charlist, random.randint(4, 10)))
            if temp2 not in seen3:
                seen3[temp2] = True
                data2.append(temp2)

        # Ascending Order
        data3 = self.mergeSort(data.copy())
        
        # Descending Order
        data4 = data3[::-1]

        # String with missing values and duplicate values
        num6 = round(size * 0.2)
        temp6 = data[ : num6]
        data5 = [None if i >= num6 and random.random() < self.percentage else data[i] for i in range(size)]
        data6 = [random.choice(temp6) if i >= num6 and random.random() < self.percentage else data[i] for i in range(size)]

        # Strings of distinct integers
        data7 = []
        for _ in range(size):
            data7.append(str(random.randint(-2147483648, 2147483647)))

        # Strings of distinct floats between 0.0 and 1.0
        data8 = []
        for _ in range(size):
            data8.append(str(random.uniform(0.0, 1.0)))

        # Strings contain whitespace
        data9 = data.copy()
        for i in range(size):
            current = data9[i]
            for _ in range(random.randint(1,10)):
                index = random.choice(range(0,len(current)))
                current = current[:index] + " " + current[index+1:]
            data9[i] = current

        # Palindrome strings
        data10 = data.copy()
        for i in range(len(data10)):
            tempword = data10[i][:len(data10[i])//2+1]
            data10[i] = tempword + tempword[::-1]

        # Strings with repeating patterns 
        data11 = []
        for i in range(size):
            pattern_length = random.randint(1,5)
            pattern = ''.join(random.choice(alphabets) for _ in range(pattern_length))
            repititions = random.randint(1,10)
            pattern = pattern * repititions
            data11.append(pattern)
        
        # Strings with alternating upper-case and lower-case letters 
        data12 = [s[0].upper() if random.randint(0, 1) == 0 else s[0].lower() + ''.join([s[i].lower() if s[i-1].isupper() else s[i].upper() for i in range(1, len(s))]) for s in data]

        self.syntheticData.append(data)
        self.syntheticData.append(data1)
        self.syntheticData.append(data2)
        self.syntheticData.append(data3)
        self.syntheticData.append(data4)
        self.syntheticData.append(data5)
        self.syntheticData.append(data6)
        self.syntheticData.append(data7)
        self.syntheticData.append(data8)
        self.syntheticData.append(data9)
        self.syntheticData.append(data10)
        self.syntheticData.append(data11)
        self.syntheticData.append(data12)

        return self.syntheticData

    def mergeSort(self, arr):
        n = len(arr)
        size = 1
        while size < n:
            for i in range(0, n, 2*size):
                left = arr[i : i+size]
                right = arr[i+size : i+2*size]
                merged = []
                l = r = 0
                while l < len(left) and r < len(right):
                    if left[l] < right[r]:
                        merged.append(left[l])
                        l += 1
                    else:
                        merged.append(right[r])
                        r += 1
                merged += left[l:]
                merged += right[r:]
                arr[i : i+len(merged)] = merged
            size *= 2
        return arr

In [ ]:
datasize = 100000

dataGenerate = TestDataGenerator()
dataGenerate.setParams(100, 0.6)
SyntheticDataList = dataGenerate.generateData(datasize)

In [ ]:
len(SyntheticDataList)

In [ ]:
SyntheticDataNames = ["Normal", "Long String", "Special String", "Ascending Order", "Descending Order", 
                      "String with missing values", "String with duplicate values",
                     "Strings of distinct integers","Strings of distinct floats",
                      "Strings contain whitespace","Palindrome strings",
                      "Strings with repeating patterns"]

In [ ]:
synSearchList = []
for datalist in SyntheticDataList:
    num = round(len(datalist) * 0.3)
    templist = []
    seen = {}
    # Choose from list
    while len(templist) < num:
        tempdata = random.choice(datalist)
        if tempdata not in seen:
            seen[tempdata] = True
            templist.append(tempdata)
    # Generate new values
    falsenum = round(len(datalist) * 0.2)
    for _ in range(falsenum):
        fdata = ''.join(random.choice(string.ascii_lowercase) for _ in range(random.randint(8, 12))) 
        templist.append(fdata)
    synSearchList.append(templist)

Test the Bloom Filter with different number of hash functions

In [ ]:
def BloomFilterFPR(datalist, searchdata, fpr, hashnum):
    testBF = BloomFilterSet()
    testBF.initVariables(len(datalist), fpr, hashFunctions[:hashnum])

    searchRes = []
    for word in datalist:
        testBF.insertElement(word)
    
    for ws in searchdata:
        searched = testBF.searchElement(ws)
        searchRes.append(searched)
    
    return searchRes

In [ ]:
for i in range(1, len(hashFunctions)+1):
    searchRes = BloomFilterFPR(SyntheticDataList[0], synSearchList[0], 0.01, i)
    index = int(len(searchRes) * 0.6)
    res1, res2 = searchRes[:index], searchRes[index:]
    
    count = 0
    for r1 in res1:
        if r1 == False:
            count += 1
    for r2 in res2:
        if r2 == True:
            count += 1
    
    tempFPR = count / len(synSearchList[0])
    print("FPR using %d hash functions:"%i, tempFPR)

#### Insert Time Taken

In [ ]:
def InsertTime2(classname, datalist):
    # Remove Duplicates
    seen = {}
    uniqueDatas = []
    for e in datalist:
        if e not in seen:
            seen[e] = True
            uniqueDatas.append(e)
    
    algorithm = classname()
    if str(classname) == str(BloomFilterSet):
        algorithm.initVariables(len(datalist), 0.0001, hashFunctions[:5])

    InsertStart = timeit.default_timer()
    res = []
    for word in uniqueDatas:
        # Remove Missing Values
        if word is not None:
            inserted = algorithm.insertElement(word)
            res.append(inserted)
    InsertEnd = timeit.default_timer()
    InsertTime = InsertEnd - InsertStart
    print("Insert time taken is :", InsertTime)
    return algorithm, res

In [ ]:
SEQSyn, BSTSyn, RBTSyn, TTSTSyn, BTSyn = [], [], [], [], []
InsertRes1, InsertRes2, InsertRes3, InsertRes4, InsertRes5 = [], [], [], [], []
for i in range(len(SyntheticDataList)):
    print("For", SyntheticDataNames[i], "Case: ")
    print("Sequential Search: ")
    SEQ, res1 = InsertTime2(SequentialSearchSet, SyntheticDataList[i])
    SEQSyn.append(SEQ)
    InsertRes1.append(res1)
    print("Binary Search Tree: ")
    BST, res2 = InsertTime2(BinarySearchTreeSet, SyntheticDataList[i])
    BSTSyn.append(BST)
    InsertRes2.append(res2)
    print("Red Black Balanced Search Tree: ")
    RBT, res3 = InsertTime2(BalancedSearchTreeSet, SyntheticDataList[i])
    RBTSyn.append(RBT)
    InsertRes3.append(res3)
    print("2-3 Balanced Search Tree: ")
    TTST, res4 = InsertTime2(TwoThreeBSTSet, SyntheticDataList[i])
    TTSTSyn.append(TTST)
    InsertRes4.append(res4)
    print("Bloom Filter: ")
    BT, res5 = InsertTime2(BloomFilterSet, SyntheticDataList[i])
    BTSyn.append(BT)
    InsertRes5.append(res5)
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

In [ ]:
SyntheicInsertResS = []
for i in range(len(SyntheticDataList)):
    SyntheicInsertList = [InsertRes1[i], InsertRes2[i], InsertRes3[i], InsertRes4[i], 
                          InsertRes5[i]]
    SyntheicInsertRes = CheckRes(SyntheicInsertList)
    SyntheicInsertResS.append(SyntheicInsertRes)
SyntheicInsertResS

### Search Time Taken

In [ ]:
def SearchTime2(obj, searchdata):
    res = []
    SearchStart = timeit.default_timer()
    for ws in searchdata:
        if ws is not None:
            searched = obj.searchElement(ws)
        else:
            searched = None
        res.append(searched)
    SearchEnd = timeit.default_timer()
    SearchTime = SearchEnd - SearchStart
    print("Search time taken is :", SearchTime)
    return res

In [ ]:
SearchRes1, SearchRes2, SearchRes3, SearchRes4, SearchRes5 = [], [], [], [], []
for i in range (len(synSearchList)):
    print("For", SyntheticDataNames[i], "Case: ")
    print("Sequential Search: ")
    sres1 = SearchTime2(SEQSyn[i], synSearchList[i])
    SearchRes1.append(sres1)
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("Binary Search Tree: ")
    sres2 = SearchTime2(BSTSyn[i], synSearchList[i])
    SearchRes2.append(sres2)
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("Red Black Balanced Search Tree: ")
    sres3 = SearchTime2(RBTSyn[i], synSearchList[i])
    SearchRes3.append(sres3)
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("2-3 Balanced Search Tree: ")
    sres4 = SearchTime2(TTSTSyn[i], synSearchList[i])
    SearchRes4.append(sres4)
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("Bloom Filter: ")
    sres5 = SearchTime2(BTSyn[i], synSearchList[i])
    SearchRes5.append(sres5)
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("                                                 ")

In [ ]:
SyntheicSearchResS = []
for i in range(len(SyntheticDataList)):
    SyntheicSearchList = [SearchRes1[i], SearchRes2[i], SearchRes3[i], SearchRes4[i], 
                          SearchRes5[i]]
    SyntheicSearchRes = CheckRes(SyntheicSearchList)
    SyntheicSearchResS.append(SyntheicSearchRes)
SyntheicSearchResS

In [ ]:
SyntheicSearchResS = []
for i in range(len(SyntheticDataList)):
    SyntheicSearchList = [SearchRes1[i], SearchRes2[i], SearchRes3[i], SearchRes4[i], 
                          SearchRes5[i]]
    for j in range(1, len(SyntheicSearchList)):
        for k in range(len(SyntheicSearchList[j])):
            if SyntheicSearchList[j][k] != SearchRes1[i][k]:
                SyntheicSearchResS.append([False, j, k])
# SyntheicSearchResS

In [ ]:
print("The False-Positive-Rate of Bloom Filter in Syntheic Data:", 
      len(SyntheicSearchResS) / (datasize * len(SyntheticDataList)))

## Visualization

In [ ]:
%pip install matplotlib

### For Real Data - Insertion

In [ ]:
# experimental  data 
# inserting function

import numpy as np
import pylab as pb

def plotOfInserting(datadict, colourdict, size):

    PlotI = pb.subplot(111)
    PlotI.set_yscale('log')
    PlotI.set_xlabel('The Number of Times')
    PlotI.set_ylabel('Time Taken')
    PlotI.set_title('Inserting time for real data')

    for classname, colour in colour_dict.items():
        
        aryx = np.arange(1,size+1)
        aryy = np.zeros(size)
        
        for key in datadict.keys():
            tempdata = datadict[key]
            for i in range(1,size+1):
                print(i)
                InstantiateTime = 0
                InsertStart = timeit.default_timer()
                for j in range(i):
                    InstantiateStart = timeit.default_timer()
                    algorithm = classname()
                    if str(classname) == str(BloomFilterSet):
                        algorithm.initVariables(len(tempdata), 0.01, hashFunctions)
                    InstantiateEnd = timeit.default_timer()
                    InstantiateTime += InstantiateEnd - InstantiateStart
                    seen = {}
                    for word in tempdata:
                        if word is not None and word not in seen:
                            seen[word] = True
                            inserted = algorithm.insertElement(word)
                InsertEnd = timeit.default_timer()
                InsertTime = (InsertEnd - InsertStart) - InstantiateTime
                aryy[i-1] += InsertTime
        
        aryy /= len(datadict)
        PlotI.plot(aryx,aryy,c=colourdict[classname])

    pb.show()

colour_dict = {
    SequentialSearchSet: 'purple',
    BinarySearchTreeSet: 'pink',
    BalancedSearchTreeSet: 'green',
    BloomFilterSet: 'blue'
    }

plotOfInserting(files, colour_dict, 25)

### For Real Data - Searching

In [ ]:
def plotOfSearching(datadict, colourdict, size):
    
    PlotS = pb.subplot(111)
    PlotS.set_yscale('log')
    PlotS.set_xlabel('The Number of Times')
    PlotS.set_ylabel('Time Taken')
    PlotS.set_title('Search time for real data')

    for classname, colour in colour_dict.items():
        aryx = np.arange(1,size+1)
        aryy = np.zeros(size)

        algorithm = classname()
        if str(classname) == str(BloomFilterSet):
            algorithm.initVariables(len(tempdata), 0.01, hashFunctions)

        for key in datadict.keys():
            tempdata = datadict[key]
            seen = {}
            for word in tempdata:
                if word is not None and word not in seen:
                    seen[word] = True
                    inserted = algorithm.insertElement(word)

            for i in range(1,size+1):
                SearchStart = timeit.default_timer()
                for j in range(i):
                    for word in wordsforsearch:
                        searched = algorithm.searchElement(word)
                SearchEnd = timeit.default_timer()
                SearchTime = (SearchEnd - SearchStart)
                aryy[i-1] += SearchTime
        
        aryy /= len(datadict)
        PlotS.plot(aryx,aryy,c=colourdict[classname])

    pb.show()

plotOfSearching(files, colour_dict, 100)

In [ ]:
def RealInsertTs(classname, datas):
    algorithm = classname()
    if str(classname) == str(BloomFilterSet):
        algorithm.initVariables(len(datas), 0.01, hashFunctions)

    seen = {}
    InsertStart = timeit.default_timer()
    for word in datas:
        if word is not None and word not in seen:
            seen[word] = True
            inserted = algorithm.insertElement(word)
    InsertEnd = timeit.default_timer()
    InsertTime = InsertEnd - InsertStart
    return algorithm, InsertTime


rInsertTS1, rInsertTS2, rInsertTS3, rInsertTS4, rInsertTS5 = 0, 0, 0, 0, 0
Seqs, Bsts, Basts, Ttsts, Bfs = [], [], [], [], []
IterationTimes = 10
RealInsertTimes = []
for key in files.keys():
    Algs1s, Algs2s, Algs3s, Algs4s, Algs5s = [], [], [], [], []
    for _ in range(IterationTimes):
        Algs1, realInsertT1 = RealInsertTs(SequentialSearchSet, files[key])
        rInsertTS1 += realInsertT1
        Algs1s.append(Algs1)
        Algs2, realInsertT2 = RealInsertTs(BinarySearchTreeSet, files[key])
        rInsertTS2 += realInsertT2
        Algs2s.append(Algs2)
        Algs3, realInsertT3 = RealInsertTs(BalancedSearchTreeSet, files[key])
        rInsertTS3 += realInsertT3
        Algs3s.append(Algs3)
        Algs4, realInsertT4 = RealInsertTs(TwoThreeBSTSet, files[key])
        rInsertTS4 += realInsertT4
        Algs4s.append(Algs4)
        Algs5, realInsertT5 = RealInsertTs(BloomFilterSet, files[key])
        rInsertTS5 += realInsertT5
        Algs5s.append(Algs5)
        RTInsertTimes = [rInsertTS1 / IterationTimes, rInsertTS2 / IterationTimes, 
                        rInsertTS3 / IterationTimes, rInsertTS4 / IterationTimes, 
                        rInsertTS5 / IterationTimes]
    RealInsertTimes.append(RTInsertTimes)
    Seqs.append(Algs1s)
    Bsts.append(Algs2s)
    Basts.append(Algs3s)
    Ttsts.append(Algs4s)
    Bfs.append(Algs5s)

In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(1, 1, figsize=(10, 10))
labels = ["Sequencial", "BinarySearchTree", "RedBlackSearchTree", "2-3SearchTree", "BloomFilter"]
colors = ["#9400D3", "#FF1493", "#FFA500", "#99CC66", "#00FFFF"]
rys = [list(x) for x in zip(*RealInsertTimes)]
filenames = ["test1-mobydick.txt", "test2-warpeace.txt", "test3-dickens.txt"]

for i in range(5):
    axs.plot(filenames, rys[i], label=labels[i], c=colors[i])
axs.set_xlabel("Cases")
axs.set_ylabel("Time Taken")
axs.set_title("Inserting Performance")
axs.set_yscale("log")
axs.legend()

plt.tight_layout()
plt.savefig('RealInsertTimes.png')
plt.show()

In [ ]:
def RealSearchTs(obj, searchdata):
    SearchStart = timeit.default_timer()
    for ws in searchdata:
        if ws is not None:
            searched = obj.searchElement(ws)
        else:
            searched = None
    SearchEnd = timeit.default_timer()
    SearchTime = SearchEnd - SearchStart
    return SearchTime

rSearchTS1, rSearchTS2, rSearchTS3, rSearchTS4, rSearchTS5 = 0, 0, 0, 0, 0
RealSearchTimes = []
for i in range(3):
    RTSearchTimes = []
    for j in range(IterationTimes):
        searchT1 = RealSearchTs(Seqs[i][j], wordsforsearch)
        rSearchTS1 += searchT1
        searchT2 = RealSearchTs(Bsts[i][j], wordsforsearch)
        rSearchTS2 += searchT2
        searchT3 = RealSearchTs(Basts[i][j], wordsforsearch)
        rSearchTS3 += searchT3
        searchT4 = RealSearchTs(Ttsts[i][j], wordsforsearch)
        rSearchTS4 += searchT4
        searchT5 = RealSearchTs(Bfs[i][j], wordsforsearch)
        rSearchTS5 += searchT5
        RTSearchTimes = [rSearchTS1 / IterationTimes, rSearchTS2 / IterationTimes, 
                        rSearchTS3 / IterationTimes, rSearchTS4 / IterationTimes, 
                        rSearchTS5 / IterationTimes]
    RealSearchTimes.append(RTSearchTimes)

In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(1, 1, figsize=(10, 10))
rys = [list(x) for x in zip(*RealSearchTimes)]
filenames = ["test1-mobydick.txt", "test2-warpeace.txt", "test3-dickens.txt"]

for i in range(5):
    axs.plot(filenames, rys[i], label=labels[i], c=colors[i])
axs.set_xlabel("Cases")
axs.set_ylabel("Time Taken")
axs.set_title("Inserting Performance")
axs.set_yscale("log")
axs.legend()

plt.tight_layout()
plt.savefig('RealSearchTimes.png')
plt.show()

### For Synthetic Data

In [ ]:
dataGenerator = TestDataGenerator()
dataGenerator.setParams(100, 0.6)
SyntheticDatasT = dataGenerator.generateData(100000)

In [ ]:
def InsertTimeTest(classname, datalist):
    # Remove Duplicates
    seen = {}
    uniqueDatas = []
    for e in datalist:
        if e not in seen:
            seen[e] = True
            uniqueDatas.append(e)
    
    algorithm = classname()
    if str(classname) == str(BloomFilterSet):
        algorithm.initVariables(len(datalist), 0.01, hashFunctions)

    InsertStart = timeit.default_timer()

    for word in uniqueDatas:
        # Remove Missing Values
        if word is not None:
            inserted = algorithm.insertElement(word)
    InsertEnd = timeit.default_timer()
    InsertTime = InsertEnd - InsertStart
    return algorithm, InsertTime

In [ ]:
def SynInsertT(ObjFunction, ObjSyn, timeList, IterateNum, data):
    tempInsertTimes = []
    tempObjs = []
    tempTimes = 0
    for _ in range(IterateNum):
        Obj, t = InsertTimeTest(ObjFunction, data)
        tempObjs.append(Obj)
        tempTimes += t
    ObjSyn.append(tempObjs)
    timeList.append(tempTimes / IterateNum)
    return ObjSyn, timeList

In [ ]:
SEQs, BSTs, RBTs, TTSTs, BFs = [], [], [], [], []
InsertTimes = []
IterateN = 3

for data in SyntheticDatasT:
    tempTs = []
    SEQs, tempTs = SynInsertT(SequentialSearchSet, SEQs, tempTs, IterateN, data)
    BSTs, tempTs = SynInsertT(BinarySearchTreeSet, BSTs, tempTs, IterateN, data)
    RBTs, tempTs = SynInsertT(BalancedSearchTreeSet, RBTs, tempTs, IterateN, data)
    TTSTs, tempTs = SynInsertT(TwoThreeBSTSet, TTSTs, tempTs, IterateN, data)
    BFs, tempTs = SynInsertT(BloomFilterSet, BFs, tempTs, IterateN, data)
    print(tempTs)
    InsertTimes.append(tempTs)

In [ ]:
import matplotlib.pyplot as plt
names = ["Normal", "LongStr", "SpecialStr", "Ascend", "Descend", "Nones", 
         "Duplicates","StrInts","StrFloats","Spaces","Palindrome","Repeats"]
labels = ["Sequencial", "BinarySearchTree", "RedBlackSearchTree", "2-3SearchTree", "BloomFilter"]
colors = ["#9400D3", "#FF1493", "#FFA500", "#99CC66", "#00FFFF"]

In [ ]:
fig, axs = plt.subplots(5, 1, figsize=(10, 10))
ys = [list(x) for x in zip(*InsertTimes)]

for i in range(5):
    axs[i].plot(names, ys[i], label=labels[i], c=colors[i])
    axs[i].set_xlabel("Cases")
    axs[i].set_ylabel("Time Taken")
    axs[i].set_title("Inserting Performance")
    axs[i].set_yscale("log")
    axs[i].legend()

plt.tight_layout()
plt.savefig('SynInsertTimes.png')
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(10, 10))
ys = [list(x) for x in zip(*InsertTimes)]

for i in range(5):
    axs.plot(names, ys[i], label=labels[i], c=colors[i])
axs.set_xlabel("Cases")
axs.set_ylabel("Time Taken")
axs.set_title("Inserting Performance")
axs.set_yscale("log")
axs.legend()

plt.tight_layout()
plt.savefig('SynInsertTimes2.png')
plt.show()

In [ ]:
synSearchDatas = []
for datalist in SyntheticDatasT:
    num = round(len(datalist) * 0.3)
    templist = []
    seen = {}
    # Choose from list
    while len(templist) < num:
        tempdata = random.choice(datalist)
        if tempdata not in seen:
            seen[tempdata] = True
            templist.append(tempdata)
    # Generate new values
    falsenum = round(len(datalist) * 0.2)
    for _ in range(falsenum):
        fdata = ''.join(random.choice(string.ascii_lowercase) for _ in range(random.randint(8, 12))) 
        templist.append(fdata)
    synSearchDatas.append(templist)

In [ ]:
def SearchTimeTest(obj, searchdata):
    SearchStart = timeit.default_timer()
    for ws in searchdata:
        if ws is not None:
            searched = obj.searchElement(ws)
    SearchEnd = timeit.default_timer()
    SearchTime = SearchEnd - SearchStart
    return SearchTime

In [ ]:
def SynSearchT(Objs, data, IterateN, tempSearchT):
    searchTSum = 0
    for i in range(IterateN):
        searchTSum += SearchTimeTest(Objs[i], data)
    tempSearchT.append(searchTSum / IterateN)
    return tempSearchT

In [ ]:
SearchTimes = []
for i in range(len(synSearchDatas)):
    tempSearchT = []
    tempSearchT = SynSearchT(SEQs[i], synSearchDatas[i], IterateN, tempSearchT)
    tempSearchT = SynSearchT(BSTs[i], synSearchDatas[i], IterateN, tempSearchT)
    tempSearchT = SynSearchT(RBTs[i], synSearchDatas[i], IterateN, tempSearchT)
    tempSearchT = SynSearchT(TTSTs[i], synSearchDatas[i], IterateN, tempSearchT)
    tempSearchT = SynSearchT(BFs[i], synSearchDatas[i], IterateN, tempSearchT)
    print(tempSearchT)
    SearchTimes.append(tempSearchT)

In [ ]:
fig, axs = plt.subplots(5, 1, figsize=(10, 10))
ys = [list(x) for x in zip(*SearchTimes)]

for i in range(5):
    axs[i].plot(names, ys[i], label=labels[i], c=colors[i])
    axs[i].set_xlabel("Cases")
    axs[i].set_ylabel("Time Taken")
    axs[i].set_title("Searching Performance")
    axs[i].set_yscale("log")
    axs[i].legend()

plt.tight_layout()
plt.savefig('SynSearchTimes.png')
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(10, 10))
ys = [list(x) for x in zip(*SearchTimes)]

for i in range(5):
    axs.plot(names, ys[i], label=labels[i], c=colors[i])
axs.set_xlabel("Cases")
axs.set_ylabel("Time Taken")
axs.set_title("Searching Performance")
axs.set_yscale("log")
axs.legend()

plt.tight_layout()
plt.savefig('SynSearchTimes2.png')
plt.show()

In [ ]:
# Testing for different input sizes

InputSizes = [1000,5000,10000,50000,100000,200000,500000]

IterateN = 3
def TestInputSize(DataStructure):
    InsertTimes = []
    SearchTimes = []
    print(f"{DataStructure}")
    for size in InputSizes:
        print(size)
        SEQs1, tempTs1 = [], []
        dataGenerator1 = TestDataGenerator()
        dataGenerator1.setParams(100, 0.6)
        SyntheticDatasT1 = dataGenerator1.generateData(size)

        SEQs1, tempTs1 = SynInsertT(DataStructure, SEQs1, tempTs1, IterateN, SyntheticDatasT1[0])
        InsertTimes.append((size, tempTs1[0]))
        print((size, tempTs1[0]))

        tempSearchT1 = []
        synSearchDatas = []

        for datalist in SyntheticDatasT1:
            num = round(len(datalist) * 0.3)
            templist = []
            seen = {}
            # Choose from list
            while len(templist) < num:
                tempdata = random.choice(datalist)
                if tempdata not in seen:
                    seen[tempdata] = True
                    templist.append(tempdata)

            # Generate new values
            falsenum = round(len(datalist) * 0.2)
            for _ in range(falsenum):
                fdata = ''.join(random.choice(string.ascii_lowercase) for _ in range(random.randint(8, 12))) 
                templist.append(fdata)
            synSearchDatas.append(templist)
        
        tempSearchT1 = SynSearchT(SEQs1[0], synSearchDatas[0], IterateN, tempSearchT1)
        
        print((size,tempSearchT1[0]))
        SearchTimes.append((size,tempSearchT1[0]))
    
    return(InsertTimes, SearchTimes)

seqInsert, seqSearch = TestInputSize(SequentialSearchSet)[0], TestInputSize(SequentialSearchSet)[1]
bstInsert, bstSearch = TestInputSize(BinarySearchTreeSet)[0], TestInputSize(BinarySearchTreeSet)[1]
llrbInsert, llrbSearch = TestInputSize(BalancedSearchTreeSet)[0], TestInputSize(BalancedSearchTreeSet)[1]
b23Insert, b23Search = TestInputSize(TwoThreeBSTSet)[0], TestInputSize(TwoThreeBSTSet)[1]
bfInsert, bfSearch = TestInputSize(BloomFilterSet)[0], TestInputSize(BloomFilterSet)[1]

insertLst = [seqInsert, bstInsert, llrbInsert, b23Insert, bfInsert]
searchLst = [seqSearch, bstSearch, llrbSearch, b23Search, bfSearch]

In [ ]:
# Graphing insert performance
import matplotlib.pyplot as plt
lst = insertLst

xs = InputSizes

ys = []
for item in lst:
    ys_sub = [data[1] for data in item]
    ys.append(ys_sub)

print(ys)

labels = ["Sequential", "BinarySearchTree", "RedBlackSearchTree", "2-3SearchTree", "BloomFilter"]
colors = ["#9400D3", "#FF1493", "#FFA500", "#99CC66", "#00FFFF"]

fig, axs = plt.subplots(1, 1, figsize=(10, 10))

for i in range(5):
    axs.plot(xs, ys[i], label=labels[i], c=colors[i])

axs.set_xlabel("Input Size")
axs.set_ylabel("Time Taken (seconds)")
axs.set_title("Insert Performance for Random Strings")
axs.set_yscale("log")
axs.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Graphing search performance
import matplotlib.pyplot as plt
lst = searchLst

xs = InputSizes

ys = []
for item in lst:
    ys_sub = [data[1] for data in item]
    ys.append(ys_sub)

print(ys)

labels = ["Sequential", "BinarySearchTree", "RedBlackSearchTree", "2-3SearchTree", "BloomFilter"]
colors = ["#9400D3", "#FF1493", "#FFA500", "#99CC66", "#00FFFF"]

fig, axs = plt.subplots(1, 1, figsize=(10, 10))

for i in range(5):
    axs.plot(xs, ys[i], label=labels[i], c=colors[i])

axs.set_xlabel("Input Size")
axs.set_ylabel("Time Taken (seconds)")
axs.set_title("Insert Performance for Random Strings")
axs.set_yscale("log")
axs.legend()

plt.tight_layout()
plt.show()